# City Shadows: Building Height Validation
This notebook validates the estimated building heights mapped via `mapping_buildings.ipynb` against OSM building height data.

## I. Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error, mean_squared_error
import requests
import time
from tqdm import tqdm 

# pip installed openpyxl, seaborn

Define paths

In [ ]:
excel_path = "dataset_manual.xlsx"
geojson_path = "buildings_dataset_updated_real_mean.geojson"
osm_ids_path = "osm_ids.txt"
osm_heights_path = "osm_heights.csv"

## II. Format geojson and excel files

A. Helper Functions

In [ ]:
# Separate values by comma or new line
def split_column(value, delimiter):
    if pd.isna(value):
        return []
    return [v.strip() for v in str(value).split(delimiter) if v.strip()]

B. EXCEL File

In [ ]:
# Read the excel file
df_excel = pd.read_excel(excel_path, sheet_name="Building")

# Separate values by comma or new line
df_excel["OSM ID"] = df_excel["OSM ID"].astype(str)
df_excel['Category'] = df_excel['Category'].apply(lambda x: split_column(x, ','))
df_excel['Height'] = df_excel['Height'].apply(lambda x: split_column(x, ','))
df_excel['File Names'] = df_excel['File Names'].apply(lambda x: split_column(x, '\n'))

# Drop columns
df_excel = df_excel.drop(columns=["Category", "Source", "File Names", "Building Name"])

print(df_excel)

C. GEOJSON File

In [ ]:
# Load GeoJSON data
with open(geojson_path, "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Extract osm_id, name, and height from GeoJSON
records = []
for feature in geojson_data["features"]:
    props = feature["properties"]
    osm_id = props.get("osm_id")
    name = props.get("name")
    height = props.get("height")
    
    if osm_id in df_excel["OSM ID"].values:
        records.append({"OSM ID": osm_id, "Building Name": name, "Height": height})

# Create df_geojson
df_geojson = pd.DataFrame(records)

print(df_geojson)

## III. Evaluate estimated heights with manually identified heights

In [ ]:
# Merge Excel and GeoJSON dataframes
merged_df = pd.merge(df_excel, df_geojson, on="OSM ID", suffixes=('_excel', '_geojson'))

print(merged_df)

A. Spearman Rank Correlation
- To measure how well the order of estimated heights matches the order of actual heights.
- To tell whether taller buildings are consistently estimated to be taller.

In [ ]:
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt

# Prepare comparison data
spearman_data = []

for _, row in merged_df.iterrows():
    predicted = row["Height_geojson"]
    actuals = row["Height_excel"]

    if isinstance(actuals, list) and len(actuals) > 0:
        try:
            actual_floats = [float(a) for a in actuals if str(a).strip() != ""]
        except ValueError:
            continue

        closest_actual = min(actual_floats, key=lambda x: abs(x - predicted))
        spearman_data.append((predicted, closest_actual))

# Unpack as lists (not tuples!)
predicted_heights, actual_heights = zip(*spearman_data)
predicted_heights = list(predicted_heights)
actual_heights = list(actual_heights)

# Compute Spearman correlation
spearman_corr, p_value = spearmanr(predicted_heights, actual_heights)

print(f"Spearman Rank Correlation: {spearman_corr:.4f}")
print(f"P-value: {p_value:.4f}")

# Visualization
plt.figure(figsize=(4, 4))  # Square aspect ratio
sns.regplot(
    x=actual_heights,
    y=predicted_heights,
    scatter_kws={'s': 50},
    line_kws={'color': 'green'}
)
plt.title("Spearman Correlation: Predicted vs Actual Heights")
plt.xlabel("Actual Height (m)")
plt.ylabel("Predicted Height (m)")
plt.grid(True)

# Determine maximum value to set symmetrical square limits
max_val_x = max(actual_heights)
max_val_y = max(predicted_heights)

# Set limits: both start at 0 and end at max_val (ensures no negatives)
plt.xlim(0, max_val_x+10)
plt.ylim(0, max_val_y+10)

# Optional: Add y = x reference line
#plt.plot([0, max_val_y], [0, max_val_x], color='red', linestyle='--', label='Perfect Prediction')
#plt.legend()

plt.tight_layout()
plt.show()

B. Mean Absolute Error / Root Mean Squared Error
- For accuracy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Merge Excel and GeoJSON dataframes
merged_df = pd.merge(df_excel, df_geojson, on="OSM ID", suffixes=('_excel', '_geojson'))

# Prepare comparison lists
comparison_data = []

for _, row in merged_df.iterrows():
    height_geojson = row["Height_geojson"]  # predicted
    height_excel = row["Height_excel"]  # actual list

    # Ensure valid height list
    if isinstance(height_excel, list) and len(height_excel) > 0:
        try:
            height_excel_floats = [float(h) for h in height_excel if str(h).strip() != ""]
        except ValueError:
            continue

        closest_actual = min(height_excel_floats, key=lambda x: abs(x - height_geojson))

        comparison_data.append({
            "Building Name": row.get("Building Name", ""),
            "Predicted Height": height_geojson,
            "Actual Height": closest_actual
        })

# Convert to DataFrame and sort by predicted height
df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values(by="Actual Height").reset_index(drop=True)

# Calculate errors
mae = mean_absolute_error(df_comparison["Actual Height"], df_comparison["Predicted Height"])
rmse = np.sqrt(mean_squared_error(df_comparison["Actual Height"], df_comparison["Predicted Height"]))

print(f"Mean Absolute Error (MAE): {mae:.2f} meters")
# print(f"Root Mean Squared Error (RMSE): {rmse:.2f} meters")

# Visualization
plt.figure(figsize=(8, 4))
x = range(len(df_comparison))

plt.plot(x, df_comparison["Predicted Height"], label="Predicted Height", color="blue", marker='o')
plt.plot(x, df_comparison["Actual Height"], label="Actual Height", color="red", marker='x')

plt.title("Building Height Comparison (Sorted by Actual Height)")
plt.xlabel("Buildings (sorted)")
plt.ylabel("Height (meters)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## IV. Evaluate estimated heights with OSM heights

A. Extract osm ids to osm_ids.txt

In [ ]:
# Load GeoJSON
with open(geojson_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract osm_ids
osm_ids = []
for feature in data.get("features", []):
    props = feature.get("properties", {})
    height = props.get("height")
    if height is not None:
        osm_id = props.get("osm_id")
        osm_ids.append(str(osm_id))

# Write to text file
with open(osm_ids_path, "w", encoding="utf-8") as f:
    for osm_id in osm_ids:
        f.write(osm_id + "\n")

print(f"✅ Saved {len(osm_ids)} OSM IDs to: {osm_ids_path}")

B. Generate csv of osm_ids with their corresponding heights (if there are)

In [ ]:
# Load OSM IDs
with open(osm_ids_path, "r") as f:
    osm_ids = [line.strip() for line in f if line.strip().isdigit()]

# Load already processed if output exists
if os.path.exists(osm_heights_path) and os.path.getsize(osm_heights_path) > 0:
    try:
        df_existing = pd.read_csv(osm_heights_path)
        processed_ids = set(df_existing["osm_id"].astype(str))
    except pd.errors.EmptyDataError:
        df_existing = pd.DataFrame(columns=["osm_id", "height"])
        processed_ids = set()
else:
    df_existing = pd.DataFrame(columns=["osm_id", "height"])
    processed_ids = set()

def get_osm_height(osm_type, osm_id):
    query = f"""
    [out:json];
    {osm_type}({osm_id});
    out tags;
    """
    url = "https://overpass-api.de/api/interpreter"
    try:
        response = requests.post(url, data={"data": query})
        response.raise_for_status()
        elements = response.json().get("elements", [])
        if elements:
            return elements[0]["tags"].get("height")
    except Exception as e:
        print(f"Error for ({osm_id}): {e}")
    return None

# Open file in append mode
with open(osm_heights_path, "a", newline="", encoding="utf-8") as f_out:
    for osm_id in tqdm(osm_ids, desc="Fetching heights"):
        if osm_id in processed_ids:
            continue

        height = get_osm_height("way", osm_id)
        # Append result as a new line to the file
        f_out.write(f"{osm_id},{height if height else ''}\n")
        f_out.flush()
        time.sleep(1)

C. CSV File (Ground truth)

In [ ]:
# Load the CSV into a DataFrame
csv_path = "osm_heights.csv"
df_csv = pd.read_csv(csv_path)

# Ensure osm_id is a string
df_csv["osm_id"] = df_csv["osm_id"].astype(str)

# Keep only rows with valid (non-null and non-empty) heights
df_csv = df_csv[df_csv["height"].notnull() & (df_csv["height"] != "")]

# Optional: convert height to float for further use
df_csv["height"] = df_csv["height"].astype(float)

# Print filtered results
print(df_csv)


D. GEOJSON File (Estimation)

In [ ]:
# Load GeoJSON data
with open(geojson_path, "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Extract osm_id, name, and height from GeoJSON
records = []
for feature in geojson_data["features"]:
    props = feature["properties"]
    osm_id = props.get("osm_id")
    name = props.get("name")
    height = props.get("height")
    
    if osm_id in df_csv["osm_id"].values:
        records.append({"osm_id": osm_id, "height": height})

# Create df_geojson
df_geojson = pd.DataFrame(records)

print(df_geojson)

E. Merge csv and geojson into one df

In [ ]:
# Ensure osm_id is string for reliable merge
df_csv["osm_id"] = df_csv["osm_id"].astype(str)
df_geojson["osm_id"] = df_geojson["osm_id"].astype(str)

# Merge the two DataFrames on 'osm_id'
merged_df = pd.merge(df_csv, df_geojson, on="osm_id", suffixes=('_actual', '_estimated'))

# Drop rows with missing or invalid heights
merged_df = merged_df.dropna(subset=["height_actual", "height_estimated"])

# Convert heights to float (if not already)
merged_df["height_actual"] = merged_df["height_actual"].astype(float)
merged_df["height_estimated"] = merged_df["height_estimated"].astype(float)

# Extract values
actual_heights = merged_df["height_actual"].values
estimated_heights = merged_df["height_estimated"].values

In [ ]:
# Define 5-meter bins
bin_edges = np.arange(0, merged_df["height_actual"].max() + 5, 5)
merged_df["height_bin"] = pd.cut(merged_df["height_actual"], bins=bin_edges)

# Compute signed mean error (Actual - Estimated)
def signed_error(group):
    if len(group) == 0:
        return np.nan
    return (group["height_actual"] - group["height_estimated"]).mean()

mean_error_per_bin = merged_df.groupby("height_bin").apply(signed_error).dropna()

# Compute midpoints for x-axis labels
mean_error_per_bin.index = mean_error_per_bin.index.map(lambda interval: interval.mid)

# Plot
plt.figure(figsize=(6, 4))
plt.bar(mean_error_per_bin.index, mean_error_per_bin.values, width=4.5, color="lightcoral", edgecolor="black")
plt.axhline(0, color="black", linestyle="--")
plt.xlabel("OSM Height (m)")
plt.ylabel("OSM Height - Estimated Height (m)")
plt.grid(True, axis='both', zorder=1)
plt.xticks(rotation=0)
plt.yticks(np.arange(-25, 301, 25))
plt.tight_layout()
plt.show()

In [ ]:
# Define 5-meter bins
bin_edges = np.arange(0, merged_df["height_actual"].max() + 5, 5)
merged_df["height_bin"] = pd.cut(merged_df["height_actual"], bins=bin_edges)

# Compute MAE only for non-empty groups
def safe_mae(group):
    if len(group) == 0:
        return np.nan
    return mean_absolute_error(group["height_actual"], group["height_estimated"])

mae_per_bin = merged_df.groupby("height_bin").apply(safe_mae).dropna()

# Compute midpoints for bin labels
mae_per_bin.index = mae_per_bin.index.map(lambda interval: interval.mid)

plt.figure(figsize=(8, 4))
plt.bar(mae_per_bin.index, mae_per_bin.values, width=4.5, color="skyblue", edgecolor="black")
plt.title("Height Estimation Error by Building Height Range (5m Bins)")
plt.xlabel("Actual Height (m)")
plt.ylabel("MAE (m)")
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

E. Spearman Rank Corellation

In [ ]:
# Compute Spearman Rank Correlation
corr, p_value = spearmanr(actual_heights, estimated_heights)
print(f"Spearman Rank Correlation: {corr:.4f}")
print(f"P-value: {p_value:.4f}")

# Visualization
plt.figure(figsize=(6, 4))  # Square aspect ratio
sns.regplot(
    x=actual_heights,
    y=estimated_heights,
    scatter_kws={'s': 10, 'alpha': 0.5},
    line_kws={'color': 'green'}
)

# Add 1:1 reference line (red)
min_val = min(min(actual_heights), min(estimated_heights))
max_val = max(max(actual_heights), max(estimated_heights))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='1:1 Line')

plt.xlabel("OSM Height (m)")
plt.ylabel("Estimated Height (m)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

F. Mean Absolute Error (MAE)

In [ ]:
# Compute MAE
mae = mean_absolute_error(actual_heights, estimated_heights)
print(f"MAE: {mae:.2f}")


# Visualization
sorted_df = merged_df.sort_values("height_actual").reset_index(drop=True)

plt.figure(figsize=(6, 4))
#plt.plot(sorted_df["height_actual"], label="Actual", color="red", linewidth=2)
#plt.plot(sorted_df["height_estimated"], label="Estimated", color="blue", linestyle="--", linewidth=2)
plt.scatter(sorted_df.index, sorted_df["height_actual"], label="OSM height", color="red", alpha=0.5, s=10)
plt.scatter(sorted_df.index, sorted_df["height_estimated"], label="Estimated height", color="blue", alpha=0.5, s=10)
plt.xlabel("Building Index")
plt.ylabel("Height (m)")
plt.legend()
plt.grid(True)

# Determine maximum value to set symmetrical square limits
max_val = max(actual_heights)
plt.xlim(0, max_val+10)
plt.ylim(0, max_val+10)
plt.tight_layout()
plt.show()

In [ ]:
errors = estimated_heights - actual_heights
plt.figure(figsize=(6, 4))
plt.hist(errors, bins=50, color="orange", edgecolor="black")
plt.axvline(0, color='black', linestyle='--')
plt.title("Distribution of Height Estimation Errors")
plt.xlabel("Error (Estimated - Actual, m)")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()